# <font color='#A7F432'><u>Product Sales - Visualization</u></font>

### Dependencies

In [17]:
#!pip install pandas
#!pip install plotly
#!pip install dash
#!pip install dash_bootstrap_components

In [18]:
import pandas as pd
pd.set_option('max_rows',20)
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"

In [19]:
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc

### Get Data

In [20]:
sales_ts = pd.read_csv('sample.csv')
df = pd.DataFrame(sales_ts.groupby(['Date','region']).agg({'product_1': 'sum', 'product_2': 'sum', 'product_3': 'sum'}))
new_sale_data = df.reset_index()

### Data Processing

In [21]:
#get data in cleaned time series format for region and product
def process_data(data,region='US', product='product_1'):
    sale_ts_reg = data[data['region']==region]
    final_dataset = sale_ts_reg
    df=pd.DataFrame(final_dataset,columns=['Date',product])
    df.set_index('Date')
    return df

### Generate Line Graph using Plotly

In [22]:
def fig_world_trend(region='US', product='product_1'):
    df = process_data(data=new_sale_data,region=region, product = product)
    df.head(10)
    yaxis_title = "Sales"
    fig = px.line(df, y=product, x=df['Date'], title='Monthly sales trend for {} - {}'.format(region, product),height=600,color_discrete_sequence =['maroon'])
    fig.update_layout(title_x=0.5,plot_bgcolor='#F2DFCE',paper_bgcolor='#F2DFCE',xaxis_title="Month",yaxis_title=yaxis_title)
    return fig

### Generate Bar Graph using Plotly

In [23]:
def fig_world_bar(region='US', product='product_1'):
    df = process_data(data=new_sale_data,region=region, product = product)
    df.head(10)
    yaxis_title = "Sales"
    fig = px.bar(df, x='Date', y=product)
#    fig = px.line(df, y=product, x=df['Date'], title='Monthly sales trend for {} - {}'.format(region, product),height=600,color_discrete_sequence =['maroon'])
    fig.update_layout(title_x=0.5,plot_bgcolor='#F2DFCE',paper_bgcolor='#F2DFCE',xaxis_title="Month",yaxis_title=yaxis_title)
    return fig

## **<font color='#FF00CC'><u>DASH APP</u></font>**

In [24]:
external_stylesheets = [dbc.themes.BOOTSTRAP]

In [25]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = 'Product Sales Dashboard'

### Page Header

In [26]:
colors = {
    'background': '#111111',
    'bodyColor':'#F2DFCE',
    'text': '#7FDBFF'
}
def get_page_heading_style():
    return {'backgroundColor': colors['background']}


def get_page_heading_title():
    return html.H1(children='Product Sales - Dashboard',
                                        style={
                                        'textAlign': 'center',
                                        'color': colors['text']
                                    })

def get_page_heading_subtitle():
    return html.Div(children='Visualize Product Sales data generated from sources all over the world.',
                                         style={
                                             'textAlign':'center',
                                             'color':colors['text']
                                         })

def generate_page_header():
    main_header =  dbc.Row(
                            [
                                dbc.Col(get_page_heading_title(),md=12)
                            ],
                            align="center",
                            style=get_page_heading_style()
                        )
    subtitle_header = dbc.Row(
                            [
                                dbc.Col(get_page_heading_subtitle(),md=12)
                            ],
                            align="center",
                            style=get_page_heading_style()
                        )
    header = (main_header,subtitle_header)
    return header

### Select Country dropdown

In [27]:
def get_country_list():
    return new_sale_data['region'].unique()

def create_dropdown_list(cntry_list):
    dropdown_list = []
    for cntry in sorted(cntry_list):
        tmp_dict = {'label':cntry,'value':cntry}
        dropdown_list.append(tmp_dict)
    return dropdown_list

def get_country_dropdown(id):
    return html.Div([
                        html.Label('Select Region'),
                        dcc.Dropdown(id='my-id'+str(id),
                            options=create_dropdown_list(get_country_list()),
                            value='US'
                        ),
                        html.Div(id='my-div'+str(id))
                    ])

### Select Product dropdown

In [28]:
def get_product_dropdown(id):
    return html.Div([
                        html.Label('Select Product'),
                        dcc.Dropdown(id='my-another-id'+str(id),
                            options=create_dropdown_list(['product_1', 'product_2', 'product_3']),
                            value='product_1'
                        ),
                        html.Div(id='my-another-div'+str(id))
                    ])

### Graph Container for DASH

In [29]:
def graph1():
    return dcc.Graph(id='graph1',figure=fig_world_trend('US','product_1'))

In [30]:
def graph2():
    return dcc.Graph(id='graph2',figure=fig_world_bar('US','product_1'))

### Generate APP layout

In [31]:
def generate_layout():
    page_header = generate_page_header()
    layout = dbc.Container(
        [
            page_header[0],
            page_header[1],
            dbc.Row(
                [
                    dbc.Col(get_country_dropdown(id=1),md=dict(size=4,offset=4))                    
                ]
            
            ),
            dbc.Row(
                [
                    dbc.Col(get_product_dropdown(id=1),md=dict(size=4,offset=4))                    
                ]
            
            ),
            dbc.Row(
                [                
                    
                    dbc.Col(graph1(),md=dict(size=6,offset=3))
        
                ],
                align="center",
            ),
            dbc.Row(
                [                
                    
                    dbc.Col(graph2(),md=dict(size=6,offset=3))
        
                ],
                align="center",
            ),
        ],fluid=True,style={'backgroundColor': colors['bodyColor']}
    )
    return layout

In [32]:
app.layout = generate_layout()

### Assign DASH Callbacks

In [33]:
@app.callback( Output(component_id='graph1',component_property='figure'), #line chart
               Output(component_id='graph2',component_property='figure'), #bar chart
               Input(component_id='my-id1',component_property='value'), #dropdown
               Input(component_id='my-another-id1',component_property='value')
)
def update_output_div(input_value1,input_value2):
    return fig_world_trend(input_value1, input_value2), fig_world_bar(input_value1, input_value2)

## RUN the app server

In [34]:
app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Aug/2021 21:52:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 21:52:36] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 21:52:36] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 21:52:36] "GET /_dash-component-suites/dash_core_components/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 21:52:36] "GET /_dash-component-suites/dash_core_components/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 21:52:36] "GET /_dash-component-suites/dash_core_components/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 21:52:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 21:53:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 21:53:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 21:53:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 21:54:43] "

## Screenshot of the DashBoard App

<img src="files/US_Pr1.JPG">

<img src="files/US_Pr2.JPG">

<img src="files/US_Pr3.JPG">

<img src="files/JAPAC_Pr2.JPG">

<img src="files/TMEA_Pr3.JPG">